In [3]:
pip install opencv-python

In [4]:
import os
import cv2

In [31]:
categories = ["1Hundrednote","2Hundrednote","5Hundrednote","2Thousandnote","Fiftynote","Tennote","Twentynote"]

In [6]:
data = []
for category in categories:
    path = os.path.join('/content/drive/MyDrive/Train',category)
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [7]:
len(data)

178

In [8]:
import random
random.shuffle(data)

In [9]:
X = []
y = []

for feature, target in data:
    X.append(feature)
    y.append(target)

In [10]:
import numpy as np

X = np.array(X)
y = np.array(y)

In [11]:
X.shape

(178, 224, 224, 3)

In [12]:
X = X/255

In [13]:
import pandas as pd
y = pd.get_dummies(y)

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.2)

In [15]:
import tensorflow as tf

In [16]:
from keras.applications.vgg16 import VGG16

In [17]:
vgg = VGG16()

553476096/553467096 [==============================] - 4s 0us/step


In [18]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
model = tf.keras.Sequential()

for layer in vgg.layers[:-1]:
  model.add(layer)
  

In [20]:
for layer in model.layers:
  layer.trainable = False

In [21]:
model.add(tf.keras.layers.Dense(7, activation="softmax"))

In [22]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics="accuracy" )

In [77]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
5/5 [==============================] - 1s 171ms/step - loss: 0.3600 - accuracy: 0.9789 - val_loss: 0.7037 - val_accuracy: 0.8056
Epoch 2/50
5/5 [==============================] - 1s 148ms/step - loss: 0.3390 - accuracy: 0.9648 - val_loss: 0.6709 - val_accuracy: 0.8333
Epoch 3/50
5/5 [==============================] - 1s 149ms/step - loss: 0.3471 - accuracy: 0.9789 - val_loss: 0.7431 - val_accuracy: 0.7778
Epoch 4/50
5/5 [==============================] - 1s 147ms/step - loss: 0.3359 - accuracy: 0.9718 - val_loss: 0.6695 - val_accuracy: 0.8333
Epoch 5/50
5/5 [==============================] - 1s 147ms/step - loss: 0.3273 - accuracy: 0.9789 - val_loss: 0.7697 - val_accuracy: 0.7778
Epoch 6/50
5/5 [==============================] - 1s 149ms/step - loss: 0.3234 - accuracy: 0.9930 - val_loss: 0.6795 - val_accuracy: 0.7222
Epoch 7/50
5/5 [==============================] - 1s 149ms/step - loss: 0.3225 - accuracy: 0.9718 - val_loss: 0.6960 - val_accuracy: 0.7778
Epoch 8/50
5/5 [====

In [24]:
y_pred = model.predict(X_test)

In [25]:
#Creating framework for webapp
def classify(img):
  img = cv2.imread(img)
  img = cv2.resize(img,(224,224))
  y_pred = model.predict(img.reshape(1,224,224,3))
  y_pred = y_pred.argmax()
  output = categories[y_pred]
  return output

In [27]:
categories[1]

'2Hundrednote'

In [28]:
# import pickle
# filehandler = open("model.pkl","wb")
# pickle.dump(model,filehandler)

In [46]:
from keras.models import load_model
model_1 = load_model('keras_model.h5')
import cv2

categs = ["10_note","20_note","50_note","100_note","200_note","500_note","2000_note"]

def classify(img):
  img = cv2.imread(img)
  img = cv2.resize(img,(224,224))
  y_pred = model_1.predict(img.reshape(1,224,224,3))
  y_pred = y_pred.argmax()
  output = categs[y_pred]
  return output

classify("5.jpg")



'50_note'

In [42]:
import blosc
import numpy as np
import pickle

pickled_data = pickle.dumps(model)  # returns data as a bytes object
compressed_pickle = blosc.compress(pickled_data)

with open("vgg_model.pkl", "wb") as f:
    f.write(compressed_pickle)

INFO:tensorflow:Assets written to: ram://5a1f84e5-2ea1-4a7c-aa40-db6f51d0667b/assets


In [53]:
import bz2
import pickle
import _pickle as cPickle

# Pickle a file and then compress it into a file with extension 
def compressed_pickle(title, data):
 with bz2.BZ2File(title + '.pbz2', 'w') as f: 
   cPickle.dump(data, f)
  

In [55]:
compressed_pickle('vgg_model_1', model)

INFO:tensorflow:Assets written to: ram://0668bc4d-2564-4c32-8793-0ebdf66afcdd/assets


In [56]:
abc = tf.keras.applications.mobilenet_v2.MobileNetV2()

In [69]:
model_4 = tf.keras.Sequential()

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.floa

ValueError: ignored

In [72]:
len(abc.layers)

156

In [76]:
abc.compile(optimizer="adam", loss="categorical_crossentropy", metrics="accuracy" )